In [1]:
!pip install spacy
!pip install nltk
!python -m spacy download en_core_web_sm

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [2]:
import pandas as pd
import numpy as np
import spacy
from spacy.pipeline import EntityRecognizer
import gensim
import pickle
import nltk
from spacy import displacy
import en_core_web_sm
import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)

nlp = en_core_web_sm.load()
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
with open('./prepared.pickle', 'rb') as f:
    df = pickle.load(f)

In [4]:
df.head()

,id,label,tweet,clean_tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,1,0.0,@user when a father is dysfunctional and is s...,when father is dysfunctional and is so selfish...,"[when, father, is, dysfunctional, and, is, so,...","[father, dysfunctional, selfish, drags, kids, ...","[father, dysfunct, selfish, drag, kid, dysfunc...","[father, dysfunctional, selfish, drag, kid, dy..."
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit cannot use cause they d...,"[thanks, for, lyft, credit, can, not, use, cau...","[thanks, lyft, credit, use, cause, offer, whee...","[thank, lyft, credit, use, caus, offer, wheelc...","[thanks, lyft, credit, use, cause, offer, whee..."
2,3,0.0,bihday your majesty,bihday your majesty,"[bihday, your, majesty]","[bihday, majesty]","[bihday, majesti]","[bihday, majesty]"
3,4,0.0,#model i love u take with u all the time in ...,model love you take with you all the time in urð,"[model, love, you, take, with, you, all, the, ...","[model, love, take, time, urð]","[model, love, take, time, urð]","[model, love, take, time, urð]"
4,5,0.0,factsguide: society now #motivation,factsguide society now motivation,"[factsguide, society, now, motivation]","[factsguide, society, motivation]","[factsguid, societi, motiv]","[factsguide, society, motivation]"


**Задание 1.**  
Используя библиотеку Spacy, вывести ТОП-20 популярных NER в combine_df датасете. Какой тип NER (ORG, GPE, PERSON и тд) оказался самым популярным? (Учтите, что max_word_limit_spacy для Spacy = 1000000)
С помощью Spacy выяснить: какие персоны и организации самые обсуждаемые в train и test датасетах? вывести ТОП-20 самых популярных. Действительно ли в топ вошли только персоны и организации или есть мусор?

In [5]:
cnt = 5000
words = []
ners = []
# весь текст из колонки не обрабатывается, разбил
for i in range(1, df.shape[0]//cnt + 2):
    for ent in nlp('. '.join(df['clean_tweet'][(i-1)*cnt:i*cnt])).ents:
        words.append(ent.text)
        ners.append(ent.label_)

In [6]:
tags = pd.DataFrame({'word': words, 'ner': ners})

Топ 20 по NER

In [7]:
tags \
    .groupby('ner') \
    .count() \
    .sort_values(['word'], ascending=False) \
    .head(20)

,word
ner,
PERSON,11325
DATE,10130
GPE,5317
ORG,4788
TIME,2008
NORP,1612
CARDINAL,1160
ORDINAL,713
FAC,495


Топ PERSON и ORG - всякое попадается

In [8]:
tags[(tags['ner']=='PERSON') | (tags['ner']=='ORG')] \
    .groupby('word') \
    .count() \
    .sort_values(['ner'], ascending=False) \
    .head(20)

,ner
word,
urð,391
ó¾,181
bihday,164
hea,153
hu,88
bing bong,78
app,75
gop,74
¼ð,60


**Задание 2.**  
Используя библиотеку nltk, вывести ТОП-20 популярных NER в combine_df датасете. Какой тип NER (ORG, GPE, PERSON и тд) оказался самым популярным? Для данного задания используем ограничение на количество символов во входном датасете (max_word_limit_spacy = 1000000), чтобы иметь возможность сравнить результаты работы Spacy и nltk. Обратите внимание, что nltk чувствителен к регистру.
С помощью nltk выяснить: какие персоны и организации самые обсуждаемые в train и test датасетах? вывести ТОП-20 самых популярных. Действительно ли в топ вошли только персоны и организации или есть мусор?

In [9]:
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

In [10]:
def preprocess(sent):
    sent = nltk.word_tokenize(sent)
    sent = nltk.pos_tag(sent)
    return sent

In [11]:
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /home/jovyan/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [12]:
document = '. '.join(df['clean_tweet'][:10])

In [13]:
chunks = nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(document)))

In [14]:
for chunk in chunks:
    if hasattr(chunk, 'label'):
        print(chunk.label())

**что-то не так**